In [1]:
pip install transformers torch onnx openvino-dev[onnx] datasets accelerate openvino peft bitsandbytes trl -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 534.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip show transformers
!pip show accelerate

Name: transformers
Version: 4.44.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl
Name: accelerate
Version: 0.33.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft, trl


In [2]:
!pip uninstall -y transformers accelerate
!pip install transformers[torch] accelerate

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)


In [3]:
import pandas as pd
import torch
from datasets import Dataset
from random import randrange
#from peft import Loraconfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
model.save_pretrained("./my_gpt_distilled_model")
tokenizer.save_pretrained("./my_gpt_distilled_model")

('./my_gpt_distilled_model/tokenizer_config.json',
 './my_gpt_distilled_model/special_tokens_map.json',
 './my_gpt_distilled_model/vocab.json',
 './my_gpt_distilled_model/merges.txt',
 './my_gpt_distilled_model/added_tokens.json',
 './my_gpt_distilled_model/tokenizer.json')

In [6]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./my_gpt_distilled_model")
model = AutoModelForCausalLM.from_pretrained("./my_gpt_distilled_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "gpt_distilled_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=14,
                  do_constant_folding=True)

print("Model converted to ONNX format successfully.")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:


Model converted to ONNX format successfully.


In [7]:
import openvino as ov
import os

ir_model = ov.convert_model("gpt_distilled_model.onnx",
                            input=[ov.PartialShape([1, 4])])

ov.save_model(ir_model, "gpt_distilled_ir_model.xml")

print("Model converted to IR format successfully.")

print("XML file exists:", os.path.exists("gpt_distilled_ir_model.xml"))
print("BIN file exists:", os.path.exists("gpt_distilled_ir_model.bin"))

Model converted to IR format successfully.
XML file exists: True
BIN file exists: True


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/medquad.csv')

df = df[['question', 'answer']]

In [9]:
train_val_df, sample_df = train_test_split(df, test_size=0.1, random_state=42) #using only sample df as the whole dataset is very big

In [10]:
sample_df['text'] = 'Question:\n' + sample_df['question'] + '\n\nAnswer:\n' + sample_df['answer']
sample_df.drop(columns=['question','answer'], axis=1,inplace=True)
sample_df.to_csv('/content/pre_dataset.csv', index=False)

In [11]:
df = pd.read_csv('/content/pre_dataset.csv')

train = Dataset.from_pandas(df)
train_dataset_size = len(train)
print(train_dataset_size)

1642


In [12]:
train

Dataset({
    features: ['text'],
    num_rows: 1642
})

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset, DatasetDict
import torch

model_name = "./my_gpt_distilled_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

train_dataset_size = len(train)
batch_size = 2
steps_per_epoch = train_dataset_size // batch_size

target_steps = 300
num_epochs = max(3, target_steps // steps_per_epoch)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
)

class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs is not None:
            if 'loss' in logs:
                print(f"Step {state.global_step}: Training Loss: {logs['loss']:.5f}")
            else:
                print(f"Step {state.global_step}: Logs: {logs}")

class PositionIdsCapCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, **kwargs):
        # Get the model directly from the kwargs
        model = kwargs.get('model')
        if model is not None:
            # Get the maximum position id the model can handle
            max_position_id = model.config.max_position_embeddings

            # Modify the training dataset to cap position ids and token ids
            def cap_ids(example):
                # Cap input_ids to be within the vocabulary size
                example['input_ids'] = [min(id, model.config.vocab_size - 1) for id in example['input_ids']]

                # Cap position_ids to be within the maximum position embeddings
                max_position_id = model.config.max_position_embeddings - 1
                position_ids = list(range(len(example['input_ids'])))
                example['position_ids'] = [min(id, max_position_id) for id in position_ids]

                return example
            if 'trainer' in kwargs and hasattr(kwargs['trainer'], 'train_dataset'):
                dataset = kwargs['trainer'].train_dataset
                dataset = dataset.map(cap_ids) # Use cap_ids here



trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    dataset_text_field='text',
    max_seq_length=720,
    tokenizer=tokenizer,
    packing=True,
    callbacks=[PrinterCallback(),PositionIdsCapCallback()]
)

trainer.train()

trainer.save_model("./fine_tuned_gpt_distilled_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will overri

Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2858 > 1024). Running this sequence through the model will result in indexing errors


Step,Training Loss
10,3.071500
20,3.064500
30,2.953100
40,3.084500
50,3.003900
60,2.953900
70,2.765200
80,2.833200
90,2.765100
100,2.825000


Step 10: Training Loss: 3.07150
Step 20: Training Loss: 3.06450
Step 30: Training Loss: 2.95310
Step 40: Training Loss: 3.08450
Step 50: Training Loss: 3.00390
Step 60: Training Loss: 2.95390
Step 70: Training Loss: 2.76520
Step 80: Training Loss: 2.83320
Step 90: Training Loss: 2.76510
Step 100: Training Loss: 2.82500
Step 110: Training Loss: 2.80380
Step 120: Training Loss: 2.71820
Step 130: Training Loss: 2.69600
Step 140: Training Loss: 2.69530
Step 150: Training Loss: 2.75790
Step 160: Training Loss: 2.58500
Step 170: Training Loss: 2.55160
Step 180: Training Loss: 2.64770
Step 190: Training Loss: 2.58610
Step 200: Training Loss: 2.50290
Step 210: Training Loss: 2.36840
Step 220: Training Loss: 2.38240
Step 230: Training Loss: 2.46490
Step 240: Training Loss: 2.66600
Step 250: Training Loss: 2.48410
Step 260: Training Loss: 2.42050
Step 270: Training Loss: 2.30440
Step 280: Training Loss: 2.25680
Step 290: Training Loss: 2.34350
Step 300: Training Loss: 2.26690
Step 310: Training 

In [15]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt_distilled_model")
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt_distilled_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "fine_tuned_gptneo_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=14,
                  do_constant_folding=True)

print("Fine-tuned model converted to ONNX format successfully.")

Fine-tuned model converted to ONNX format successfully.


In [20]:
from openvino.tools import mo
ov_model = mo.convert_model("fine_tuned_gptneo_model.onnx", input=[('input_ids', [1, -1]), ('attention_mask', [1, -1])], output=['output'])
from openvino.runtime import serialize
serialize(ov_model, "fine_tuned_gpt_distilled_ir_model.xml")
print("Fine-tuned model converted to IR format successfully.")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
Fine-tuned model converted to IR format successfully.


In [22]:
import openvino.runtime as ov
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt_distilled_model")

core = ov.Core()
model = core.read_model("fine_tuned_gpt_distilled_ir_model.xml")
compiled_model = core.compile_model(model, "CPU")

infer_request = compiled_model.create_infer_request()

def generate_text(prompt, max_length=200): # The maximum length is 200; the answer may be short.
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.numpy()
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.numpy()

    for _ in range(max_length):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}

        results = infer_request.infer(inputs)
        logits = results[next(iter(results))]

        next_token_id = np.argmax(logits[0, -1, :])

        input_ids = np.concatenate([input_ids, np.array([[next_token_id]])], axis=1)
        attention_mask = np.concatenate([attention_mask, np.array([[1]])], axis=1)

        if next_token_id == tokenizer.eos_token_id:
            break

    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

prompt = "What is (are) Schwannomatosis ?"
generated_text = generate_text(prompt)
print("Generated text:", generated_text)

#True answer: Schwannomatosis is a rare form of neurofibromatosis that is primarily characterized by multiple schwannomas (benign tumors of the nervous system) in the absence of bilateral (affecting both sides) vestibular schwannomas. Signs and symptoms of the condition vary based on the size, location and number of schwannomas but may include pain; numbness; tingling; and/or weakness in the fingers and toes. Inherited forms of the disorder account for only 15 percent of all cases. In some of these families, schwannomatosis is caused by changes (mutations) in the SMARCB1 or LZTR1 genes; in other cases, the exact underlying cause is unknown. When inherited, the condition is passed down in an autosomal dominant manner with highly variable expressivity and reduced penetrance. Treatment is based on the signs and symptoms present in each person but may include medications and/or surgery.

Generated text: What is (are) Schwannomatosis?

Answer:
Schwannomatosis is a condition that affects the body's ability to digest food and digest food. It is caused by a buildup of fat in the blood and a buildup of fat in the blood. The body's ability to digest food and digest food is impaired by the buildup of fat in the blood. The body's ability to digest food and digest food is impaired by the buildup of fat in the blood. The body's ability to digest food and digest food is impaired by the buildup of fat in the blood. The body's ability to digest food and digest food is impaired by the buildup of fat in the blood. The body's ability to digest food and digest food is impaired by the buildup of fat in the blood. The body's ability to digest food and digest food is impaired by the buildup of fat in the blood. The body's ability to digest food and digest food is impaired by the buildup of fat in the blood. The body's ability to digest
